In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
find_best_layer_strict.py

変更点:
 - サンプル数を2000に増加
 - Hard Negative（専門用語の羅列など）を追加して難易度アップ
 - 5-Fold Cross-Validation (交差検証) を導入し、スコアの安定性を評価
 - AccuracyだけでなくAUCスコアも確認
"""

import os
import random
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from transformers import AutoTokenizer, AutoModelForCausalLM

# ==========================================
# 1. 厳密設定
# ==========================================
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
N_SAMPLES = 2000  # データ数を大幅増
SEED = 42

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)

# ==========================================
# 2. 高難易度データセット生成
# ==========================================
def generate_rigorous_dataset(model, tokenizer, n_samples):
    print(f"Generating {n_samples} samples for RIGOROUS scanning...")
    
    # --- Positive (真の創造性) ---
    prompts_pos = [
        "Write a short poem using only simple words:",
        "Describe a complex emotion using a metaphor:",
        "Compose a mysterious story opening:",
        "Propose a groundbreaking research idea combining quantum physics and LLMs:",
        "Suggest a novel method to train AI without using any human data:",
    ]

    # --- Negative 1: Boring (退屈) ---
    prompts_boring = [
        "State a dry fact about history:",
        "Explain a basic grammatical rule:",
        "Describe the process of boiling water:",
    ]
    
    # --- Negative 2: Hard Negative (クリシェ・専門用語の羅列) ---
    # これらは「難しそうな単語」を含むが、創造的ではない（Label 0）
    # 層によってはこれらを「創造的」と勘違いしやすい
    prompts_jargon = [
        "Write a sentence full of technical buzzwords but with no meaning:",
        "Generate a generic corporate mission statement:",
        "Write a standard, cliché introduction for a machine learning paper:",
    ]

    data = []
    labels = []

    # 50% Positive
    for _ in tqdm(range(int(n_samples * 0.5)), desc="Generating Positives"):
        prompt = random.choice(prompts_pos)
        inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            out = model.generate(**inputs, max_new_tokens=64, do_sample=True, temperature=0.95)
        text = tokenizer.decode(out[0], skip_special_tokens=True).replace(prompt, "").strip()
        data.append(text)
        labels.append(1)

    # 50% Negative (Mix of Boring and Hard Negatives)
    for _ in tqdm(range(int(n_samples * 0.5)), desc="Generating Negatives"):
        if random.random() < 0.5:
            # Boring
            prompt = random.choice(prompts_boring)
            temp = 0.1
        else:
            # Hard Negative (Jargon/Cliché)
            prompt = random.choice(prompts_jargon)
            temp = 0.8 # 少し多様性を持たせて紛らわしくする
            
        inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            out = model.generate(**inputs, max_new_tokens=64, do_sample=True, temperature=temp)
        text = tokenizer.decode(out[0], skip_special_tokens=True).replace(prompt, "").strip()
        data.append(text)
        labels.append(0)

    return data, labels

# ==========================================
# 3. 特徴量抽出 (メモリ管理強化)
# ==========================================
def extract_all_layers(model, tokenizer, texts, batch_size=8):
    model.eval()
    all_features = []
    
    print("Extracting features from ALL layers...")
    
    # バッチ処理
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i : i+batch_size]
        batch_texts = [t if t.strip() else "empty" for t in batch_texts]
        
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=128).to(DEVICE)
        
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)
        
        mask = inputs.attention_mask.unsqueeze(-1) # [Batch, Seq, 1]
        
        batch_layer_features = []
        
        # 全層ループ
        for h_state in outputs.hidden_states:
            h_state = h_state.float() # [Batch, Seq, Dim]
            
            # Global Average Pooling (Masked)
            masked_h = h_state * mask
            sum_h = masked_h.sum(dim=1)
            count = mask.sum(dim=1)
            mean_h = sum_h / count.clamp(min=1e-9) # [Batch, Dim]
            
            batch_layer_features.append(mean_h.cpu().numpy())
            
        # [Layers, Batch, Dim] -> [Batch, Layers, Dim]
        batch_layer_features = np.stack(batch_layer_features, axis=1)
        all_features.append(batch_layer_features)

    return np.concatenate(all_features, axis=0)

# ==========================================
# 4. 厳密評価ループ (5-Fold CV)
# ==========================================
print(f"Loading Model: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, device_map="auto"
)

# A. データ生成
texts, labels = generate_rigorous_dataset(model, tokenizer, N_SAMPLES)
labels = np.array(labels)

# B. 特徴量抽出 [Samples, Layers, Dim]
X_all = extract_all_layers(model, tokenizer, texts)
num_layers = X_all.shape[1]

# C. 評価
results = []
print("\n=== 🛑 Running 5-Fold Cross-Validation per Layer ===")
print(f"{'Layer':<5} | {'Mean Acc':<10} | {'Std Dev':<10} | {'Status'}")
print("-" * 55)

# 交差検証の設定 (5分割, シャッフルあり)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

for layer_i in range(num_layers):
    X_layer = X_all[:, layer_i, :]
    
    # ロジスティック回帰 (Linear Probe)
    clf = LogisticRegression(max_iter=2000, solver='liblinear')
    
    # 5回学習・評価を行う
    scores = cross_val_score(clf, X_layer, labels, cv=skf, scoring='accuracy')
    
    mean_acc = scores.mean()
    std_acc = scores.std()
    
    status = ""
    if mean_acc > 0.95: status = "⭐⭐ Excellent"
    elif mean_acc > 0.90: status = "⭐ Good"
    
    results.append({
        "Layer": layer_i,
        "Mean_Acc": mean_acc,
        "Std_Acc": std_acc
    })
    
    print(f"{layer_i:<5} | {mean_acc:.4f}     | ±{std_acc:.4f}   | {status}")

# ==========================================
# 5. 可視化・保存
# ==========================================
df = pd.DataFrame(results)

# グラフ描画 (エラーバー付き)
plt.figure(figsize=(12, 6))
plt.errorbar(df["Layer"], df["Mean_Acc"], yerr=df["Std_Acc"], fmt='-o', ecolor='red', capsize=5)
plt.title(f"Rigorous Layer Separability Analysis (5-Fold CV)\nModel: {MODEL_NAME}")
plt.xlabel("Layer Index")
plt.ylabel("Mean Accuracy")
plt.grid(True)
plt.savefig("rigorous_layer_scan.png")

# ベストレイヤー選定 (平均精度が高く、かつ安定しているもの)
# ソート: 平均精度降順
best_row = df.sort_values(by="Mean_Acc", ascending=False).iloc[0]
best_layer = int(best_row["Layer"])

print("\n" + "="*50)
print(f"🏆 FINAL VERDICT (Strict Mode)")
print(f"Best Layer: {best_layer}")
print(f"Accuracy  : {best_row['Mean_Acc']:.4f} (±{best_row['Std_Acc']:.4f})")
print("="*50)
print("Please update 'TARGET_LAYER' in your probe script to this value.")